# Tensorflow的使用笔记

In [2]:
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import math
import time
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

## 1. 张量
tf.Tensor类

1. 张量不可修改，只能新建
2. 张量有统一的dtype，比如tf.float32, tf.int32, tf.string, tf.bool等。它们都是tf.dtypes.DType类(查看类方法：https://www.tensorflow.org/api_docs/python/tf/dtypes/DType)
3. 张量有axis，每个axis上的元素数量相同

创建张量的函数有：
1. tf.constant()
2. tf.ones()
3. tf.zeros()

创建的时候可以设定dtype

In [ ]:
# scalar. This will be an int32 tensor by default; see "dtypes" below.
# scalar的shape是空的
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

In [ ]:
# 可以认为一维张量是列向量
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

In [ ]:
# 可以在创建张量的时候设定dtype
rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)

In [ ]:
# 3维张量
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])
# 4维张量
rank_4_tensor = tf.zeros([3, 2, 4, 5])

#### tensor和array互相转化

tf.Tensor -> np.array
1. gen_array = gen_tensor.numpy()
2. gen_array = np.array(gen_tensor)

np.array -> tf.Tensor
1. gen_tensor = tf.constant(gen_array)  
由于gen_array是可变的数据，变动gen_array可能会导致gen_tensor自动变化（也可能没变，很奇怪）。见下面这个trunk
2. gen_array -> list -> tf.Tensor
可以从list构造tf.Tensor，即：  
gen_list = gen_array.tolist()  
gen_tensor = tf.constant(gen_list)  
这样即使变动gen_list，gen_tensor也不会随之改变

In [ ]:
# 多跑几次，有时候是[1.,2.,3.]，有时又会是[7.,2.,3.]
a = np.array([1.0, 2.0, 3.0])
b = tf.constant(a)
a[0] = 7.0
print(b)

#### 索引  
标量索引 -> 移除轴  
切片:索引 -> 保留轴  
对于多轴tensor也是如此。虽然多轴tensor可以不在每个轴上都索引，但推荐对每个轴都索引，获得想要的数据格式（标准化代码）  
比如：  
    rank_3_tensor[0, :, :] #取出了第一个二维tensor，移除了第一个轴，结果是二维tensor
    rank_3_tensor[0:1, :, :] #也取出了第一个二维tensor，但保留了第一个轴，结果是三维tensor

In [ ]:
print(rank_3_tensor)

In [ ]:
# 在最后一维作标量索引，所以最后一维被移除了。原倒数第二维度成了最后一维，变成了横向
print(rank_3_tensor[:, :, -1])

In [ ]:
# 在最后一维作切片索引，最后一维没有移除
print(rank_3_tensor[:, :, -1:])

#### tensor的shape及其转换
1. 用.shape属性 或 .get_shape()方法取出gen_tensor的shape。返回的是tf.TensorShape类。shape是每个轴的长度
2. tensor的轴的排列顺序（从左到右）一般从全局到局部；如果二维表示地话，从左到右一般是大的纵向，到小的纵向，到最小的纵向，只有最后一维是横向
    1. 最后一个轴一般是features，所以shape[-1]一般是每个样本元素的features个数（样本元素组合成为单个样本）
    2. 从倒数第二个轴开始，到正数第二个轴，一般是空间位置信息
    3. 第一个轴一般是批次index

很多原始数据的轴排列方式并不一定符合上述论述，可能需要对应调整  
TensorFlow 采用 C 样式的“行优先”内存访问顺序，即最右侧的索引值递增对应于内存中的单步位移

reshape  
  
TensorFlow 采用 C 样式的“行优先”内存访问顺序，即
1. 最右侧的轴的索引值递增对应于内存中的单步位移。设最右侧的维度为n_right_1st
2. 第二右侧的轴的索引值递增对应内存中的n_right_1st步位移  
3. 以此类推，一直到第一个轴

所以如果数据：
1. 生产数据为(batch_size, space_info, features_dims)的结构。  
这里features_dims是一个样本元素的特征维度。对一个样本来说，其特征维度是所有样本元素的特征维度之和。  
2. reshape到(batch_size, -1)的二维结构。  
此reshape过程把batch维度之外的轴，全部移除。这样一来，每一个样本（共有batch_size个样本），其所有特征在内存中是连续放置的。  

总结：要定义清楚样本是什么。reshape的结果是：一个样本一行，其所有特征随着列横向排列，它们在内存中连续
注意：reshape只是重新组织指向内存地址的方式，数据在内存中的放置并不会改边。所以reshape很快。但reshape在返回赋值的时候是复制数据到新的内存的。
reshape无法换轴（横纵转置意味着数据在内存中的排列方式改变）

In [ ]:
print(rank_3_tensor)

In [ ]:
print(tf.reshape(rank_3_tensor, [3*2, 5]))

transpose  
  
要改变数据在内存中的具体放置，需要用tf.transpose。tf.transpose返回内存连续的、转置之后的数组  
tf.transpose作用在二维数组上（设数组宽度为w，高度为h），就是横纵对换  
--> 从左上角(0,0)开始，往右单步位移，往下是w步位移，此为原数组
--> 往右是w步位移，往下是单步位移，此为转置后数组

如何理解三个及以上轴的transpose, 设tensor的形状为（b，h，w）
--> 从（0，0，0）开始，第三轴往右是单步位移，第二轴往下是w步位移，第一轴往下是h*w步位移，此为原数组
--> 从（0，0，0）开始，第三轴往右是单步位移，第二轴往下是h*w步位移，第一轴往下是w步位移，此为(1,0,2)转置数组
--> 从（0，0，0）开始，第三轴往右是h*w步位移，第二轴往下是w步位移，第一轴往下是单步位移，此为(2,1,0)转置数组
以此类推，其实就是步长的对换

In [ ]:
print(tf.transpose(rank_3_tensor, perm=(1,0,2)))

#### tensor的dtype及其转换
1. 用.dtype属性取出gen_tensor的dtype。返回的是tf.DType类

cast  
  
可以改变tensor的数据类型，使用tf.cast。需制定转变后的数据类型

In [ ]:
rank_3_tensor.dtype

In [ ]:
rank_3_tensor_fl = tf.cast(rank_3_tensor, dtype=tf.float16)
rank_3_tensor_fl.dtype

#### tensor运算及broadcast

In [ ]:
x = tf.constant([1,2,3])
x = tf.reshape(x, [3,1])
y = tf.range(1, 5)
print(x, "\n")
print(y, "\n")
print(tf.multiply(x, y))

In [ ]:
# 使用tf.broadcast_to()展示tensor按照shape广播的结果
print(tf.broadcast_to(x, [3, 4]))

#### 转化为tensor
tf.convert_to_tensor()方法可以np.array, TensorShape, list和tf.Variable转化为tf.Tensor

#### 不规则张量 ragged tensor
不规则张量是tf.RaggedTensor类  
创建： tf.ragged.constant()  
shape：.shape属性返回shape，None说明该轴长度未知

In [ ]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]
ragged_tensor = tf.ragged.constant(ragged_list)
print(ragged_tensor)

In [ ]:
ragged_tensor.shape

#### 字符串张量
tf.string是一种原子类型的dtype，类似tf.int32或tf.float64，无法像python字符串一样索引。tf.string的长度并不是张量的一个轴。
创建：tf.constant()

In [ ]:
# 标量scalra字符串张量
# 前缀b代表字节字符串
scalar_string_tensor = tf.constant("Gray wolf")
print(scalar_string_tensor)
# 使用unicode字符串，加前缀u
# unicode_string_tensor = tf.constant(u"Gray wolf")

In [ ]:
# 字符串向量
tensor_of_strings = tf.constant(["Gray wolf", "Quick brown fox", "Lazy dog"])
print(tensor_of_strings)

字符串操作  
由于tf.string是原子类型的dtype，要使用tf.strings中的方法来对tensor of string作操作，比如：
1. 分割：tf.strings.split() -> 从tensor of string中分割每个字符串元素到token。如果分割的长度不一，则返回tf.RaggedTensor
2. 转换数字：tf.strings.to_number() -> 从tensor of string中转化数字字符串到数字
3. 转换ASC码或unicode码：
    1. tf.strings.bytes_split() -> 从tensor of string中分割每个字符串元素到字节。如果分割的长度不一，则返回tf.RaggedTensor
    2. tf.io.decode_raw() # 详见tf.io模块
    3. tf.strings.unicode_split() -> 从tensor of string中分割每个unicode字符串元素到字节。如果分割的长度不一，则返回tf.RaggedTensor
    4. tf.strings.unicode_decode() -> 从tensor of string中分割每个unicode字符串元素到unicode码。如果分割的长度不一，则返回tf.RaggedTensor

In [ ]:
byte_strings = tf.strings.bytes_split(tensor_of_strings)
print(byte_strings)

In [ ]:
tf.strings.unicode_decode(tensor_of_strings, 'UTF-8')

#### 稀疏张量 sparse tensor
sparse tensor只储存非0元素的位置indices，非0元素的值values(与indices长度相同，且顺序一致)，稀疏张量完整表示时的dense_shape  
以上三个参数indices、values、dense_shape都以tf.Tensor类传入tf.sparse.SparseTensor()函数，即构造了相应的sparse tensor  
tf.sparse.to_dense(gen_sparse_tensor)即可得到稀疏特征对应的稠密特征

In [ ]:
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(sparse_tensor)

In [ ]:
print(tf.sparse.to_dense(sparse_tensor))

## 2.变量
tf.Variable类

1. 创建：tf.Variable变量需要提供一个张量tf.Tensor作为初始值。此初始值的dtype和shape确定了tf.Variable的dtype和shape
2. 运算：大部分tf.Tensor的运算也可以用在tf.Variable上。比如tf.argmax。具体运算的时候，是使用变量的支持张量执行运算
3. 转换：tf.Tensor、ndarray等(tf.convert_to_tensor(var), var.numpy())
4. 赋值：gen_variable.assign()重新分配张量，有如下特点：
    1. 调用 assign（通常）不会分配新张量，而会重用现有张量的内存。
    2. assign的时候不能改变shape
    3. 还有gen_variable.assign_add(), gen_variable.assign_sub()等方法
4. 复制、reshape、命名
    1. 复制：从变量a创建变量b b = tf.Variable(a) 会复制支持张量，两个变量不共享内存空间
    2. 重构：reshape是先复制再reshape，并不会reshape原本的

In [ ]:
# 创建变量
sup_tensor = tf.constant([[1,2],[3,4]])
var = tf.Variable(sup_tensor)
print(var, '\n')
print(var.dtype)
print(var.shape)

In [ ]:
# 转换为tensor或ndarray
print("as tensor: \n", tf.convert_to_tensor(var))
print('as ndarray:\n', var.numpy())

#### 命名和梯度
1. 可以在创建variable的时候，为该变量提供自定义变量名，只需设name="set_name"。变量名可以重复。在模型中，变量名会自动获得且一般不重复
2. 可以在创建variable的时候，关闭梯度，只需设trainable=False

In [ ]:
a = tf.Variable([1,2,3], name='a')
print(a.name)

## 3. 自动微分和梯度
tensorflow提供了一个tf.GradientTape()上下文管理器，来打开、关闭记录相关运算的tape。  
在BP过程中，此tape可以拿来计算涉及的node的梯度。

1. source node创建的时候发生了什么：
    1. 由于tf.Variable()中trainable参数为True，x将是个可训练变量  

  
2. 打开tape的时候发生了什么：
    tf.GradientTape(persistent=False, watch_accessed_variables=True)  
    使用watch_accessed_variables参数，以及tf.watch
    1. 自动监控所有可训练变量：GradientTape 默认（watch_accessed_variables=True）  
    将所有可训练变量（created by tf.Variable, where trainable=True）视为需要监控的 source node  
    2. 对于不可训练的张量（比如tf.constant），可以使用tape.watch()对其进行“监控”  
    还可以设定watch_accessed_variables=False，然后使用tf.watch()精确控制需要监控哪些变量  
    3. 可选persistent=True。默认persistent=False

3. 前向计算过程中发生了什么：
    1. 依赖source node的计算节点被创建。直到target node。target node一般是trainable的标量  
    2. tf.functions和tf.tensor重载后的+ - * / **等函数会创建中间值node。这些中间值node一般是tf.tensor类，带梯度，但不会有name，且不会被tape.watched_variables()显示
    3. 连接source node、中间节点、target的operation，以及assign、tape.stop_recording()截断，得到计算图  
    4. tape.reset()清除已有的计算图

4. 关闭tape的时候发生了什么：
    1. 如果persistent=False，那么默认tape在进行一次求导.gradient()之后就销毁计算图，从而节约内存
    2. 如果设定persistent=True，那么可以在同一个计算图下多次求导。在求导过程全部结束后要del tape来释放资源

如果这里调用tape.watched_variables()，可以看到所有被tape监控的source node

5. 反向计算过程中发生了什么：
    1. tape.gradient(target, sources, unconnected_gradients, output_gradients)求target对sources的偏导数
    其中：
        1. target一般是标量tensor，且一般就是loss
        2. sources可以传入列表或字典的嵌套组合。gradient返回的grad和sources具有相同的结构。
        3. unconnected_gradients可选'none'和'zero'，是指source node和target node在计算图上不联通时，返回的导数
        4. output_gradients默认为None，不操作；如果要传入的话，它是权重。gradient乘以权重之后再输出

In [ ]:
# 前向计算-tape记录
x = tf.Variable(3.0) # source node创建
with tf.GradientTape() as tape: # 打开tape
    y = x**2 # 依赖source node的节点创建。并且这里y是target node。y作为运算结果，类型一般是tensor
    # x依据tensor的计算而来，所以y是tensor
# 关闭tape

In [ ]:
# 反向计算-tape求解grad
dy_dx = tape.gradient(y, x) # 传入target node，传入source node

In [ ]:
# target node一般是标量
y

In [ ]:
# gradient的shape与source node相同
dy_dx

In [ ]:
# source node
x

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w') # w是3x2的矩阵变量
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b') # b是长度为2的变量，
x = [[1., 2., 3.]]

In [ ]:
y = x @ w + b # tensor是横向量，或者更准确如下：
# y = x @ w + tf.reshape(b, (1,2))
y

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b # 中间节点
    loss = tf.reduce_mean(y**2) # target node，也就是loss
    # tf.reduce_mean(input_tensor, axis=None, keepdims=False, name=None)
    # axis=0是第一个轴（最左边轴，二维来看是竖方向最外面轴），axis=-1是最右边轴（二维来看是横方向轴），axis=None是所有轴

In [ ]:
# 求梯度
# 传入列表结构
[dl_dw, dl_db] = tape.gradient(loss, [w, b])
# dl_dw: loss相对w矩阵的偏导数
# dl_db: loss相对b向量的偏导数

In [ ]:
# 求梯度
# 传入字典结构
my_vars = {
    'w':w,
    'b':b
}
grad = tape.gradient(loss, my_vars) # 返回的grad也是一个字典结构
grad['w']

#### 控制tape监视的内容
1. 前向计算tape创建之前建立的tf.Tensor类变量，不可训练。可以用tape.watch(tensor)来求对它的梯度
2. 前向计算tape过程中，tf.functions和tensor重载后的+ - * / **等运算，会创建中间值node。这些中间值node一般是tf.tensor类，带梯度，但不会有name，且不会被tape.watched_variables()显示
3. 前向计算tape过程中，a = b之类的python重引用不会创建中间值node，仅仅只是python变量重引用而已
4. 前向计算tape过程中，.assign()类函数也不会创建新的node。

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable, but can be wachted
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
    tape.watch(x3)
    y = (x0**2) + (x1**2) + (x2**2) + (x3**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
    print(g)

In [ ]:
tape.watched_variables()
## tensor 可以被tape.watch()监视并求梯度，但是不会被显示在tape.watched_variables()里

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1)
    y0 = tf.math.sin(x0)
    y1 = tf.nn.softplus(x1) # softplus(x) = log(exp(x) + 1)，是relu(x)的smooth版本
    y = y0 + y1
    ys = tf.reduce_sum(y)

In [ ]:
grad = tape.gradient(ys, {'x0':x0, 'x1':x1})
grad

In [ ]:
# 对中间结果求梯度。一般来说中间结果（包括target node）都是tf.tensor类，不过都是trainable的，带梯度。
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1)
    y0 = tf.math.sin(x0)
    y1 = tf.nn.softplus(x1)
    y = y0 + y1
    ys = tf.reduce_sum(y)
grad = tape.gradient(ys, {'x0':x0, 'y':y})
grad

#### 非标量target的梯度
梯度根本上是对标量的运算  
所以target node是非标量的tensor时，梯度等于target node的总和（这个总和肯定是标量），对source node的梯度

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    y = x * x # y是长度为2的tensor
    z = y * y # z是长度为2的tensor

# 由于tape的persistent设为True，我们可以多次求导
print(tape.gradient(z, x).numpy()) # grad的形状和source node一样：其实是对source node的每一个标量求偏导
print(tape.gradient(y, x).numpy())

In [ ]:
# 解除引用
del tape

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
    y0 = x**2
    y1 = 1/x

print(tape.gradient({'y0':y0, 'y1':y1}, x).numpy()) # target node是一个字典，多维target tensor
# 是target tensor的总和对source node的偏导数

In [ ]:
x = tf.Variable(2.)
with tf.GradientTape() as tape:
    y = x*[3., 4.] # target node tensor是一个长度为2的tensor
print(tape.gradient(y, x).numpy())
# 是target tensor的总和对source node的偏导数

#### 控制流
1. tf2具有动态图的体系，所以在动态执行的时候，它可以根据if-else控制语句，构建不同的计算图  
2. tf1是静态图体系，所以tf构建的计算图，在执行的时候必须确定。如果根据不同的张量值，需要数据流入不同的node，需要使用tf.cond函数

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0, name='v0')
v1 = tf.Variable(2.0, name='v1')

with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    if x > 0.0: # 根据张量值的不同情况，构建了不同的计算图 --> 动态图体系
      result = v0 # node result和v0连接(其实这里result直接指向了v0，是同一个内存对象)，没有和v1连接
    else:
      result = v1**2 # node result和v1连接（这里result创建了一个新的node，因为它经过了v1的运算），没有和v0连接

In [ ]:
result

In [ ]:
dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

In [ ]:
# tape之外创建的tf.Tensor是不带梯度的
x = tf.Variable(2.0)
for epoch in range(2):
    with tf.GradientTape() as tape:
        y = x+1
    print(type(x).__name__, ":", tape.gradient(y, x)) # 第一遍运行的时候，x还是tf.Variable类；第二遍运行的时候，x已经是tape之外定义的tf.Tensor类
    x = x + 1   # This should be `x.assign_add(1)` # x变成tf.Tensor类

In [ ]:
# 没有用tf.functions或重载后的+-*/**得到的变量，也很可能丧失梯度
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
    x2 = x**2

    # This step is calculated with NumPy，y是ndarray
    y = np.mean(x2, axis=0)
    print(type(y))
    # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
    # using `tf.convert_to_tensor`.
    # y通过隐性tf.convert_to_tensor，转化为了tensor。所以它已经不带梯度
    y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

In [ ]:
# int类型和字符串类型的tensor，就算监视了，也不带梯度
x = tf.constant(10) # x是int类型的tensor

with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
print(g.gradient(y, x))

In [ ]:
# tf.Variables.assign()函数只是对内部值的操作，所以会截断梯度
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
    # Update x1 = x1 + x0.
    x1.assign_add(x0) # x1的值从3.0更新到了3.0+0.0即3.0。
    # The tape starts recording from x1.
    y = x1**2   # y = (x1 + x0)**2 y节点并没有与x0节点建立连接

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x0)

#### 控制梯度记录
1. 在tape内部，可以用tape.stop_recording()上下文管理器。在该管理器内部所做的运算，可以产生新node，但新node和其上下游node都将不会有依赖，即计算图中的孤岛
2. 在tape内部，可以使用tf.stop_gradient()函数,精细地控制某一个node的上下游依赖。被处理的node，其与上游的依赖不变，其与下游的依赖会被解除
2. 在tape内部，可以用tape.reset()重启tape梯度记录器。这一般用于：根据不同的条件，生成不同的计算图（类似动态图？）

In [17]:
# t.stop_recording() context manager
# 其中产生的新node和输入node没有依赖，新node参与运算的更后面的node和新node直接也没有依赖
# 即：里面产生的新node在计算图中是孤岛

x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as t:
    x_sq = x * x
    with t.stop_recording():
        y_sq = y * y
    z = x_sq + y_sq

In [12]:
print(t.gradient(y_sq, y)) # y_sq和其的输入node没有依赖

None


In [18]:
grad = t.gradient(z, {'x': x, 'y': y, 'y_sq': y_sq, 'x_sq': x_sq})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])
print('dz/dx_sq:', grad['x_sq'])
print('dz/dy_sq:', grad['y_sq']) # y_sq和其下游node也没有依赖

del t

dz/dx: tf.Tensor(4.0, shape=(), dtype=float32)
dz/dy: None
dz/dx_sq: tf.Tensor(1.0, shape=(), dtype=float32)
dz/dy_sq: None


In [30]:
# tf.stop_gradient()函数：精细地控制某一个node的上下游依赖
# 被处理的node，其与上游的依赖不变，其与下游的依赖会被解除

x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as t:
    y_sq = y**2 # y_sq和上游node还有依赖
    z = x**2 + tf.stop_gradient(y_sq) # y_sq和下游node没有依赖

In [31]:
print(t.gradient(y_sq, y)) # y_sq和上游node还有依赖

tf.Tensor(6.0, shape=(), dtype=float32)


In [32]:
grad = t.gradient(z, {'x': x, 'y': y, 'y_sq': y_sq})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])
print('dz/dy_sq', grad['y_sq']) # y_sq和下游node没有依赖

del t

dz/dx: tf.Tensor(4.0, shape=(), dtype=float32)
dz/dy: None
dz/dy_sq None
